In [2]:
import numpy as np

In [1]:
from sklearn.metrics import roc_auc_score

In [13]:
import sklearn.neighbors as nb
from sklearn.preprocessing import StandardScaler

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
from sklearn.model_selection import GridSearchCV

In [5]:
import pandas as pd

In [6]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
labels = pd.read_csv('labels_training_set.csv')

In [41]:
processed = pd.read_csv('features_10.csv')

In [42]:
processed = processed.set_index('person').fillna(0)
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,05-17,05-17,6,1,...,False,False,False,Sao Paulo,Brazil,0.0,0.000000,0.0,0.0,0.0
00091926,viewed product,iPhone 6S,Computer,0,34,372,05-03,05-31,448,28,...,False,False,False,Rio Grande do Sul,Brazil,1679.0,1527.091398,0.0,4649.0,100.0
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,03-26,03-26,10,1,...,False,False,False,Minas Gerais,Brazil,1679.0,1342.333333,0.0,1679.0,849.0
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,05-17,05-26,206,10,...,True,False,False,Sao Paulo,Brazil,369.0,479.366013,379.0,2229.0,250.0
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,05-29,05-29,17,1,...,False,False,False,Minas Gerais,Brazil,2229.0,2229.000000,0.0,2229.0,2229.0


In [43]:
training_features = labels.set_index('person').join(processed)
predict_features = kaggle.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop('label',axis=1)

In [44]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,viewed product,iPhone 5s,Smartphone,1,17,23,05-22,05-31,68,9,...,False,False,False,Parana,Brazil,999.0,663.913043,459.0,999.0,269.0
6ec7ee77,visited site,Ninguno,Smartphone,0,1,0,05-28,05-28,2,1,...,False,False,False,Rio Grande do Norte,Brazil,0.0,0.000000,0.0,0.0,0.0
abe7a2fb,viewed product,iPhone 6,Smartphone,0,22,31,02-10,05-29,96,109,...,False,False,True,Sao Paulo,Brazil,1499.0,1184.161290,0.0,1589.0,179.0
34728364,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,05-18,05-27,37,9,...,True,False,False,Sao Paulo,Brazil,479.0,529.916667,0.0,849.0,60.0
87ed62de,viewed product,Samsung Galaxy J7,Computer,0,1,9,05-18,05-19,17,1,...,False,False,False,Santa Catarina,Brazil,549.0,549.000000,0.0,549.0,549.0


## Mean encoding

In [45]:
tg = category_encoders.TargetEncoder()
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [46]:
train_features_norm = StandardScaler().fit_transform(encoded_training)
predict_features_norm = StandardScaler().fit_transform(encoded_predict)

In [50]:
params={
    'n_neighbors':[5,10,25,50,100,200],
    'p':[1,2,3,4,5,6,7]
}
grid = GridSearchCV(nb.KNeighborsClassifier(),params,scoring='roc_auc',cv=2,verbose=3)

In [51]:
xtr,xte,ytr,yte=train_test_split(train_features_norm,training_labels)

In [52]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 42 candidates, totalling 84 fits
[CV] n_neighbors=5, p=1 ..............................................
[CV] ..... n_neighbors=5, p=1, score=0.6174520308140027, total=  13.4s
[CV] n_neighbors=5, p=1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.6s remaining:    0.0s


[CV] ..... n_neighbors=5, p=1, score=0.6149132138276479, total=  13.2s
[CV] n_neighbors=5, p=2 ..............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   53.4s remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
grid.score(X=xte,y=yte)